In [ ]:
import matplotlib
import numpy as np
%matplotlib widget
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import math
from IPython.display import HTML
import pandas as pd
import os, re

plt.rcParams['animation.ffmpeg_path'] = '/opt/ffmpeg/bin/ffmpeg'

LVC_path = '/misc/home/u1220/analysis_jupyter'
import sys
sys.path.remove('/home/u1220/lvd_aniso/jupiter_books') if '/home/u1220/lvd_aniso/jupiter_books' in sys.path else None
sys.path.append(LVC_path) if not LVC_path in sys.path else None

from LVC.LVCcase_cabel import CabelCase
plt.style.use('LVC')

In [ ]:
experiments_dir = '/misc/home/u1220/lvd_aniso/1D_APmu/results/id_2202'

dirs = os.listdir(experiments_dir)
dirs = [x for x in dirs if x != 'source' and  x != 'sources' and os.path.isdir("%s/%s" % (experiments_dir, x))]
# dirs = filter(lambda x: int(re.search(r'gCaL=0.(\d+)$', x).group(1)) == 25, dirs)
# dirs = filter(lambda x: int(re.search(r'slope=1\.(\d)', x).group(1)) == 4, dirs)
# dirs = filter(lambda x: int(re.search('pace_period=(\d+)', x).group(1)) < 280 and int(re.search('pace_period=(\d+)', x).group(1)) > 200, dirs)    
# dirs = filter(lambda x: int(re.search(r'mu1=0\.(\d+)', x).group(1)) == 11, dirs)
# dirs = filter(lambda x: int(re.search(r'dr=0\.(\d+)', x).group(1)) == 2, dirs)
# dirs = sorted(dirs, key=lambda x: int(re.search('(\d+)_', x).group(1)))
dirs = sorted(dirs, key=lambda x: int(re.search(r'pace_period=(\d+)', x).group(1)))

apdxs = []
dis = []

plt.close('all')

for ex_dir in dirs:
    if ex_dir != 'source' and os.path.isdir("%s/%s" % (experiments_dir, ex_dir)):
        ex_dir = "%s/%s" % (experiments_dir, ex_dir)
        print(ex_dir)        
        proj = CabelCase(project_folder=ex_dir)
        proj.read_input()
        proj.load_udat()

        
#         try:
#             proj.load_udat()
#             figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')
#             proj.draw_heatmap(plt, vmax=25)
#         except Exception:
#             pass
        
        try:
#             point_x = 125
#             x_ind = int(point_x/(proj.mult*proj.dr))
            x_ind = int(proj.udat[0].size/2)
            figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')
            plt.grid()
            plt.title("Potential in time at point %.2f" % (x_ind * proj.dr * proj.mult), fontsize=14)
            plt.xlabel('Time, ms', fontsize=14)
            plt.ylabel('Potential, mV', fontsize=14)
            plt.tight_layout()
            plt.plot(np.arange(0, proj.time, proj.dT), proj.udat[:,x_ind])
        except Exception as e:
            print(e)
        
#         x, start_t, u_level = (5, 0, 20)
#         start_t_ind = int(start_t/proj.dT)
#         times, periods = proj.calculate_periods(x=x, u_level=u_level, start=start_t_ind)
#         plt.scatter(times, periods, s=7)
#         plt.grid()
        
#         c = 40
#         (x1, x2, y1, y2) = (200, 500, 200, 500)
#         figure(num=None, figsize=((x2-x1)/c+1, (y2-y1)/c), dpi=80, facecolor='w')
#         figure(num=None, figsize=(10, 10), dpi=80, facecolor='w')

#         res = proj.get_measured_vs_stimulated(_error=.3, last_count=5)

#         plt.scatter(res[:,0], res[:,2], marker='o', color='red', s=3)
#         plt.xticks(fontsize=16)
#         plt.yticks(fontsize=16)
#         plt.title("Measured period at point 50 mm", fontsize=16)
#         plt.xlabel("Stimulation period, ms", fontsize=16)
#         plt.ylabel("Measured period, ms", fontsize=16)
#         plt.grid()
#         # plt.yticks(range(y1,y2+1,40))
#         # plt.xticks(range(x1,x2+1,40))
# #         plt.xlim(x1, x2)
# #         plt.ylim(y1, y2)
#         plt.tight_layout()

#         plt.show()
        
#         vel = proj.get_velocity(left=30, right=220, u_level=-40)
#         print(round(vel,4))
        
#         APDX = DI = 0
#         x_ind = int(x/(proj.mult*proj.dr))
#         start_t_ind = int(start_t/proj.dT)
#         times, apds, _dis = proj.calculate_APD_by_percentage_with_DI(x=x_ind, start=start_t_ind, APD_percent=.9)
#         APDX=apds[-1]
#         DI = _dis[-1]
#         print("APDX = %.1f, DI = %.2f" % (APDX, DI))
#         apdxs.append(APDX)
#         dis.append(DI)

In [ ]:
experiments_dir = '/misc/home/u1220/lvd_aniso/1D_allmodels/results/id_1689'

dirs = os.listdir(experiments_dir)
dirs = [x for x in dirs if x != 'source' and os.path.isdir("%s/%s" % (experiments_dir, x))]
dirs = sorted(dirs, key=lambda x: int(re.search('(\d+)_', x).group(1)))

for ex_dir in dirs:
    if ex_dir != 'source' and os.path.isdir("%s/%s" % (experiments_dir, ex_dir)):
        ex_dir = "%s/%s" % (experiments_dir, ex_dir)
        print(ex_dir)
        cur = "CaL50"
#         cur = re.search('\d_(.+)$', ex_dir).group(1)
        proj = CabelCase(project_folder=ex_dir)
        proj.read_input(verbose=False)                
        proj.load_udat()
        
        files = os.listdir(proj.project_folder)
        slurm_file = None

        for file in files:
            if re.search('slurm', file):
                slurm_file = "{experiment_folder}/{filename}".format(experiment_folder=proj.project_folder, filename=file)
                break

        s = open(slurm_file, 'r').read()
        stim_table = np.array(re.findall(r'period=(\d+).0 moment_stim=(\d+).0', s))
        stim_table = stim_table.astype(np.int)
        periods = stim_table[::15]        
        
        # APD90vsTIME
        x, start_t, count = (50, 0, 4)

        x_ind = int(x/(proj.mult*proj.dr))
        start_t_ind = int(start_t/proj.dT)

        times, apds, dis = proj.calculate_APD_by_percentage_with_DI(x=x_ind, start=start_t_ind, APD_percent=.9)
        _times = []
        _apds = []
        _dis = []

        for i in range(0, len(times), 15):
            _times.extend(times[i+15-count:i+15])
            _apds.extend(apds[i+15-count:i+15])
            _dis.extend(dis[i+15-count:i+15])

        figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')
        plt.title("%d last APD90s at point %d mm, slope=%.1f" % (count, x, proj.slope), fontsize=16, pad=40)
        plt.plot(times, apds, marker='.', markersize=4, linewidth=1)
#         plt.xlim([0, 150000])
        plt.xlabel("Time, ms", fontsize=16)
        plt.xticks(fontsize=16)
        plt.ylabel("APD90, ms", fontsize=16)
        plt.yticks(fontsize=16)
        plt.grid()

        ax2 = plt.gca().twiny()
        ax2.set_xticks(periods[:,1][::10])
        ax2.set_xticklabels(periods[:,0][::10], fontsize=16, rotation='vertical')
        plt.tight_layout()

        plt.savefig("../_trash/1D_instability/APD90vsTIME/%s_%d_last_APD90_at_%d_slope_%.1f.png" % (cur, count, x, proj.slope))
        # END APD90vsTIME
        
        ## DYNAMIC RESTITUTION CURVE

        figure(num=None, figsize=(10, 12), dpi=80, facecolor='w')

        i = 0
        for p in periods[:,0]:
            plt.scatter([p]*len(_apds[i:i+count]), _apds[i:i+count], color='blue', s=4)
            i += count

        plt.title("APD(T) dynamic restitution curve at point %d mm, slope=%.1f" % (x, proj.slope), fontsize=16, pad=40)
        plt.xlabel('Stimulation period, ms', fontsize=16)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.ylabel('APD90, ms', fontsize=16)
        ax = plt.gca()
        ax.set_xticks(range(150,401,10))
        plt.xticks(rotation=45)
        ax.set_yticks(range(100,401,10))
        plt.grid()
        plt.ylim([100,400])
        plt.xlim([150,400])
        plt.tight_layout()

        plt.show()
        plt.savefig("../_trash/1D_instability/APDvsPERIOD/%s_rest_curve_%d_last_APD90_at_%d_slope_%.1f.png" % (cur, count, x, proj.slope))
        ## END DYNAMIC RESTITUTION CURVE
        
        ## STEADY-STATE RESTITUTION CURVE

        figure(num=None, figsize=(7, 6), dpi=80, facecolor='w')

        nn = periods[0][0]-periods[-1][0]
        p_dis = _dis[1:count*nn]
        p_apds = _apds[1:count*nn]

        print(len(p_dis), len(p_apds))

        plt.scatter(p_dis, p_apds, marker='.')
        # print(np.transpose([p_dis, p_apds]))
        plt.title("APD restitution curve at point %d mm" % x, fontsize=16)
        plt.xlabel('DI, ms', fontsize=16)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.ylabel('APD90, ms', fontsize=16)
        plt.grid()
        plt.xlim([0,350])
        plt.ylim([100,400])
        plt.tight_layout()
        plt.show()

        plt.savefig("../_trash/1D_instability/APDvsDI/%s_APDvsDI_%d_last_APD90_at_%d_slope_%.1f.png" % (cur, count, x, proj.slope))
        ## END STEADY-STATE RESTITUTION CURVE
        
        ## PERIOD CALCULATION
        x, start_t, u_level, Nrepeat, Nbegin = (x, 0, -70, 15, 10)
        start_t_ind = int(start_t/proj.dT)
        x_ind = int(x/(proj.mult*proj.dr))

        times, periods = proj.calculate_periods(x=x_ind, start=start_t_ind, u_level=u_level)

        current_period = proj.period
        mid_periods = [[current_period, periods[0:Nbegin]]]
        periods_start = [0, Nbegin*current_period]
        current_period -= 1

        for i in range(Nbegin, len(periods), Nrepeat):
            if len(periods[i:i+Nrepeat]) == Nrepeat:
                mid_periods.append([current_period, periods[i:i+Nrepeat]])
                periods_start.append(periods_start[-1] + Nrepeat*current_period)
                current_period -= 1
            else:
                mid_periods.append([current_period, periods[len(periods) - i:]])
                periods_start.append(periods_start[-1] + Nrepeat*current_period)
                current_period -= 1
                break
        ## END PERIOD CALCULATION
        
        ## MEASURED PERIOD VS STIMULATION PERIOD
        skip_num = 15 - count

        figure(num=None, figsize=(6, 12), dpi=80, facecolor='w')

        for i in range(0, len(mid_periods) - 1):
            current_period = mid_periods[i][0]
            plt.scatter(([current_period]*len(mid_periods[i][1]))[skip_num:], mid_periods[i][1][skip_num:], marker='o', color='red', s=5)

        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.title("Measured period at point %d mm" % x, fontsize=16)
        plt.xlabel("Stimulation period, ms", fontsize=16)
        plt.ylabel("Measured period, ms", fontsize=16)
        plt.grid()
        plt.yticks(range(200,701,50))
        plt.xlim(150, 400)
        plt.ylim(200, 700)
        # plt.tight_layout()
        plt.show()
        plt.savefig("../_trash/1D_instability/MEASURED_TvsSTIM_T/%s_period_at_%d_slope_%.1f.png" % (cur, x, proj.slope))